<a href="https://colab.research.google.com/github/chaturvediajay/NLP/blob/main/spell_corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# function to tokenise words
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

In [4]:
# create a frequency table of all the words of the document
all_words = Counter(words(open('/content/drive/MyDrive/al_ml_project/NLP/big.txt').read()))

In [5]:
# check frequency of a random word, say, 'chair'
all_words['chair']

135

In [6]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [7]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

In [8]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

In [9]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

In [10]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

In [11]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

In [12]:
print(len(set(edits_one("monney"))))
print(edits_one("monney"))

336
{'mojney', 'monmney', 'mohney', 'umonney', 'monneyk', 'motney', 'monnfey', 'mqonney', 'mofnney', 'monnecy', 'mhonney', 'monnewy', 'wmonney', 'sonney', 'msnney', 'monnen', 'mowney', 'monnjy', 'monnezy', 'monnuey', 'mwonney', 'monnzey', 'monyey', 'monniey', 'monnef', 'monneey', 'monnwy', 'meonney', 'dmonney', 'monneuy', 'mongey', 'mobnney', 'moncney', 'monny', 'monneyh', 'moniey', 'monneyt', 'monneyg', 'monnney', 'xonney', 'monndey', 'monnaey', 'mjonney', 'monnny', 'monnsy', 'ionney', 'moxney', 'mfnney', 'monnhey', 'monnety', 'mconney', 'bonney', 'monnky', 'eonney', 'monvney', 'ymonney', 'monxey', 'mgonney', 'monnedy', 'monnew', 'monnxey', 'mynney', 'mznney', 'monnez', 'monnev', 'monnvey', 'onney', 'monneyp', 'monpey', 'mdonney', 'monneyf', 'mkonney', 'monneyx', 'lmonney', 'tmonney', 'mvonney', 'monneqy', 'mouney', 'monngy', 'monrey', 'monnery', 'monngey', 'konney', 'monneya', 'mgnney', 'mosnney', 'moxnney', 'donney', 'mobney', 'ponney', 'pmonney', 'monnemy', 'mfonney', 'mtonney', 'm

In [13]:
print(known(edits_one("monney")))

{'monkey', 'money'}


In [14]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'monkey', 'money'}


In [15]:
# Let's look at possible corrections of a word
print(possible_corrections("monney"))

{'monkey', 'money'}


In [16]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.0002922233626303688
5.378344097491451e-06


In [17]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [18]:
# test spell check
print(spell_check("monney"))

Did you mean money?


In [30]:
# test spell check
print(possible_corrections("emfasize"))

{'emphasize'}
